In [72]:
import requests
import json
response = requests.get("https://api.globe.gov/search/v1/measurement/protocol/measureddate/?protocols=mosquito_habitat_mapper&startdate=2020-05-31&enddate=2020-07-03&geojson=TRUE&sample=FALSE")

mosquito_data = response.json()["features"]
#mosquito_data

In [73]:
import pandas as pd
data_df_mosquito = pd.DataFrame(mosquito_data)

mosquito_list = []
for index, row in data_df_mosquito.iterrows():
    mosquito_data_dict_1 = {**data_df_mosquito.loc[index, "properties"], **data_df_mosquito.loc[index, "geometry"] }
    mosquito_list.append(mosquito_data_dict_1)


In [74]:
key_data = ["coordinates", "mosquitohabitatmapperComments", "mosquitohabitatmapperSpecies", "mosquitohabitatmapperMosquitoPupae", "mosquitohabitatmapperLarvaeCount", "mosquitohabitatmapperMosquitoEggCount"]
mosquito_data = pd.DataFrame(mosquito_list)[key_data]
mosquito_data = mosquito_data.rename(columns={"coordinates": "mcoords"})
mosquito_data

,mcoords,mosquitohabitatmapperComments,mosquitohabitatmapperSpecies,mosquitohabitatmapperMosquitoPupae,mosquitohabitatmapperLarvaeCount,mosquitohabitatmapperMosquitoEggCount
0,"[-77.196178, 39.253437]",null,null,false,null,null
1,"[-77.196178, 39.253437]",null,null,false,null,null
2,"[-77.196178, 39.253437]",null,null,false,null,null
3,"[-77.197336, 39.253415]",null,null,false,null,null
4,"[-77.197336, 39.253415]",null,null,false,null,null
...,...,...,...,...,...,...
1267,"[-80.852777, 33.973252]",null,null,false,null,null
1268,"[-95.427761, 29.781095]",null,null,false,null,null
1269,"[-95.427761, 29.781095]",null,null,false,null,null
1270,"[99.602763, 17.178488]",null,null,false,null,null


In [77]:
land_cover_url = "https://api.globe.gov/search/v1/measurement/protocol/measureddate/?protocols=land_covers&startdate=2020-05-31&enddate=2020-07-03&geojson=TRUE&sample=FALSE"
response_land_cover = requests.get(land_cover_url)

land_data = response_land_cover.json()["features"]
#land_data


In [78]:
data_df_land = pd.DataFrame(land_data)


In [79]:
land_list = []
key_land_data = ["coordinates", "landcoversMucCode"]
for index, row in data_df_land.iterrows():
    land_data_dict_1 = {**data_df_land.loc[index, "properties"], **data_df_land.loc[index, "geometry"] }
    land_list.append(land_data_dict_1)

    
land_df = pd.DataFrame(land_list)[key_land_data]
land_df = land_df.rename(columns={"coordinates": "landcoords"})
land_df

,landcoords,landcoversMucCode
0,"[39.138221, 21.611502]",M94
1,"[39.138221, 21.611502]",M91
2,"[-105.235524, 40.014562]",null
3,"[-105.258979, 40.019918]",null
4,"[40.969218, 24.229192]",M81
...,...,...
2349,"[-67.149634, 18.011173]",null
2350,"[-67.15055, 18.013892]",null
2351,"[-67.156149, 18.02027]",null
2352,"[-67.15987, 18.025726]",null


In [113]:
from math import radians, sin, cos, acos
from decimal import *
max_dist = 100
def get_dist(coordOne, coordTwo):
    
    slat = radians(coordOne[0])
    slong = radians(coordOne[1])
    elat = radians(coordTwo[0])
    elong = radians(coordTwo[1])
    dist = (sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slong - elong))
    if dist == 1.0000000000000002:
        dist = 1
    return 6371.01 * 1000 * acos(dist)
def in_range(coordOne, coordTwo):
    try:
        return abs(round(get_dist(coordOne, coordTwo))) <= max_dist
    except ValueError:
        print("death with: " + str(coordOne) + str(coordTwo))
        return False
    
    



In [112]:
get_dist([100.22076, 16.780741],[100.22076, 16.780741])

1


0.0

In [101]:
get_dist([33.169962,-96.700258], [33.165462,-96.700258])

0.9999999969157487


500.3779492185948

In [114]:
in_range([100.22076, 16.780741],[100.22076, 16.780741])

True

In [115]:
import numpy as np
import time
start_time = time.time()
land_and_mosquito_list = []

def take_smaller(s1, s2):
    if s1 > s2:
        return s1
    else:
        return s2
for index_m, row_m in mosquito_data.iterrows():
    for index_l, row_l in land_df.iterrows():
        if in_range(mosquito_data.loc[index_m, "mcoords"], land_df.loc[index_l, "landcoords"]):
            land_and_mosquito_list.append(pd.concat([row_l,row_m]))
            break;

df = pd.DataFrame(land_and_mosquito_list)

print("Total time: " + str(time.time() - start_time))
df

Total time: 192.20888590812683


,landcoords,landcoversMucCode,mcoords,mosquitohabitatmapperComments,mosquitohabitatmapperSpecies,mosquitohabitatmapperMosquitoPupae,mosquitohabitatmapperLarvaeCount,mosquitohabitatmapperMosquitoEggCount
0,"[-84.288398, 33.77668]",null,"[-84.288398, 33.77668]",This is one of my mosquito traps hanging from ...,null,false,null,null
1,"[-84.288398, 33.77668]",null,"[-84.288398, 33.77668]","This is my control mosquito trap, that Is away...",null,false,null,null
2,"[-84.288398, 33.77668]",null,"[-84.288398, 33.77668]",null,null,false,null,null
3,"[-95.419709, 29.790264]",null,"[-95.419688, 29.789362]",null,null,false,null,null
4,"[100.349522, 16.813377]",M56,"[100.349522, 16.813377]",null,null,false,null,null
...,...,...,...,...,...,...,...,...
510,"[-95.551351, 29.556863]",null,"[-95.551351, 29.556863]",null,null,false,null,null
511,"[-84.295438, 33.758803]",null,"[-84.295494, 33.757002]",I found some odd organisms in this site even t...,null,false,0,null
512,"[-80.852775, 33.974154]",null,"[-80.852777, 33.973252]",null,null,false,null,null
513,"[-80.852775, 33.974154]",null,"[-80.852777, 33.973252]",null,null,false,null,null


In [126]:
non_null_df = df[df["landcoversMucCode"]!="null"]
non_null_df

,landcoords,landcoversMucCode,mcoords,mosquitohabitatmapperComments,mosquitohabitatmapperSpecies,mosquitohabitatmapperMosquitoPupae,mosquitohabitatmapperLarvaeCount,mosquitohabitatmapperMosquitoEggCount
4,"[100.349522, 16.813377]",M56,"[100.349522, 16.813377]",null,null,false,null,null
6,"[-84.701607, 39.473094]",M43,"[-84.701578, 39.473994]",I haven’t seen any mosquitos moving around her...,null,false,null,null
7,"[-84.701607, 39.473094]",M43,"[-84.701578, 39.473994]",null,null,false,null,null
8,"[-84.701607, 39.473094]",M43,"[-84.701578, 39.473994]",I can’t dump the water out because those are m...,null,false,null,null
9,"[-86.763322, 33.436463]",M91,"[-86.763322, 33.436463]",null,null,false,0,null
...,...,...,...,...,...,...,...,...
493,"[-121.330011, 44.040259]",M71,"[-121.330011, 44.040259]",null,null,false,null,null
494,"[-97.896887, 30.369085]",M91,"[-97.896918, 30.366378]",null,null,false,null,null
505,"[-98.470841, 29.481047]",M02,"[-98.470841, 29.481047]",This habitat is a fenced off indent in the gro...,null,false,null,null
506,"[-121.314274, 44.067037]",M56,"[-121.314274, 44.067037]",null,null,false,null,null
